In [1]:
import numpy as np
import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool

In [2]:
output_notebook()

Loading BokehJS ...

### Plotting data from Pandas DataFrames

In [3]:
mpg=pd.read_csv("data/datacamp/mpg.csv")

In [4]:
mpg.columns

Index(['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'model_year', 'origin', 'name'],
      dtype='object')

In [5]:
p=figure(plot_width=600, plot_height=300, 
         x_axis_label="HP", 
         y_axis_label="MPG")
p.circle(x=mpg["horsepower"], y=mpg["mpg"], size=10)
show(p)

### The Bokeh ColumnDataSource

In [6]:
source=ColumnDataSource(mpg)
p=figure(plot_width=600, plot_height=300, 
        x_axis_label="Horsepower", 
        y_axis_label="MPG")
p.circle(x="horsepower", y="mpg", source=source)
show(p)

### Specifying tools

### Selection and non-selection glyphs

In [7]:
p = figure(plot_width=600, plot_height=300,
            x_axis_label="Year", 
            y_axis_label="Time", 
            tools="box_select")

# Add circle glyphs to the figure p with the selected and non-selected properties
p.circle(x="horsepower", y="mpg", source=source, 
         selection_color="red",
         nonselection_color="grey",
         nonselection_alpha=0.2)
show(p)

### Customizing axes

In [8]:
t=np.arange(0, 1, 0.001)
y=np.sin(2*np.pi*50*t)
p=figure(plot_width=600, plot_height=300)

p.line(t, y, line_color="Red")

p.xaxis.axis_label="Time"
p.xaxis.axis_line_color="blue"
p.xaxis.axis_line_width=3

p.yaxis.axis_label="Magnitude"
p.yaxis.major_label_text_color='orange'

p.axis.minor_tick_in=-3
p.axis.minor_tick_out=6

show(p)

## Positioning the toolbar

In [9]:
x=np.arange(0,10,0.1)
y=np.sin(x)
p=figure(plot_width=600, plot_height=300)
p.line(x,y, color="red", line_width=2)
show(p)

### Set `toolbar_location` position in `"left"` `"right"` `"above"` `"below"`
If you would like to hide the toolbar entirely, pass `None`

In [10]:
p=figure(plot_width=600, plot_height=300, toolbar_location="below")
p.line(x,y, color="red", line_width=2)
show(p)

Note that the toolbar position clashes with the default axes. In this case, setting the `toolbar_sticky` option to `False` will move the toolbar outside of the area where the axis is drawn

In [11]:
x=np.arange(0,10,0.1)
y=np.cos(x)
p=figure(plot_width=600, plot_height=300, toolbar_location="above", 
        toolbar_sticky=False)
p.line(x,y, color="DarkOrange", line_width=2)
show(p)

In [12]:
df=pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")

In [13]:
df_melt=pd.melt(df, id_vars=['Province/State', 'Country/Region', 'Lat', 
                             'Long'], var_name='date', 
                value_name='confirmed_case')
df_melt.head()

Province/State Country/Region       Lat       Long     date  confirmed_case
0            NaN    Afghanistan  33.93911  67.709953  1/22/20               0
1            NaN        Albania  41.15330  20.168300  1/22/20               0
2            NaN        Algeria  28.03390   1.659600  1/22/20               0
3            NaN        Andorra  42.50630   1.521800  1/22/20               0
4            NaN         Angola -11.20270  17.873900  1/22/20               0

In [14]:
df_melt.dtypes

Province/State     object
Country/Region     object
Lat               float64
Long              float64
date               object
confirmed_case      int64
dtype: object

In [15]:
df_melt.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', 'date',
       'confirmed_case'],
      dtype='object')

In [16]:
df_bd=df_melt[df_melt["Country/Region"]=='Bangladesh']

In [17]:
bd=df_bd.loc[:, ["date", "confirmed_case"]]
bd['daily_case']=df_bd.confirmed_case.diff().fillna(0)

In [18]:
bd.head()

date  confirmed_case  daily_case
20    1/22/20               0         0.0
295   1/23/20               0         0.0
570   1/24/20               0         0.0
845   1/25/20               0         0.0
1120  1/26/20               0         0.0

In [19]:
bd["date"]=pd.to_datetime(bd.date)

In [20]:
bd.date.min()

Timestamp('2020-01-22 00:00:00')

In [21]:
bd.date.max()

Timestamp('2021-05-15 00:00:00')

In [22]:
bd.tail()

date  confirmed_case  daily_case
130645 2021-05-11          776257      1230.0
130920 2021-05-12          777397      1140.0
131195 2021-05-13          778687      1290.0
131470 2021-05-14          779535       848.0
131745 2021-05-15          779796       261.0

In [23]:
bd.dtypes

date              datetime64[ns]
confirmed_case             int64
daily_case               float64
dtype: object

In [24]:
bd.isnull().sum()

date              0
confirmed_case    0
daily_case        0
dtype: int64

In [25]:
bd_source=ColumnDataSource(bd)

In [26]:
bd_source

ColumnDataSource(id='1940', ...)

In [28]:
hover=HoverTool(tooltips=[("date", "@date{%F}"), 
                    ("daily case", "@daily_case")], 
formatters={"@date" : "datetime"}, mode="mouse")
            

p=figure(plot_width=600, plot_height=400, x_axis_type="datetime",
        x_axis_label="Time", 
        y_axis_label="Daily infection", 
        title="Bangladesh Covid-19 daily case")
p.add_tools(hover)
p.line(x="date", y="daily_case", source=bd_source, color="DarkOrange")
show(p)